In [4]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from scipy import signal
from scipy.fft import fftshift

In [5]:
enter = pd.read_csv('../data/marking/enter.csv')

In [6]:
enter

,dt,inn,gtin,prid,operation_type,cnt
0,2021-11-22,F0C677A146366202906368AF9FCEC188,C238E7C33B56BA911F576777316EC9F2,F0C677A146366202906368AF9FCEC188,Импорт. Не ЕАЭС,6
1,2021-11-22,F0DC262C40FBA363CFCEF8005E4C6140,4D3423957ECDC40F70E37EB75A6A2D3E,F0DC262C40FBA363CFCEF8005E4C6140,Перемаркировка,10
2,2021-11-22,32D845A9CC2A5BFA9F77CAE0E9A905A5,F1931263F03E1D89A43A3633AAF9FB84,32D845A9CC2A5BFA9F77CAE0E9A905A5,Импорт. Не ЕАЭС,600
3,2021-11-22,CE816FDF82B121C67CAE9B79203B8BA1,0374D8CAC82E9F0D47C95FA13D68C7C6,CE816FDF82B121C67CAE9B79203B8BA1,Импорт. Не ЕАЭС,1860
4,2021-11-22,E34F3F6C9E49FE46C87D067306AAC29B,BBC31CA374A30B34CAFBFD027888A73D,E34F3F6C9E49FE46C87D067306AAC29B,РФ,1
...,...,...,...,...,...,...
740579,2022-11-21,6B8E111AB5B5C556C0AEA292ACA4D88B,476D3708FBD4E6B64F81D10BC28EA694,7D1B71037D27D5597BEA1D5077663541,Возврат,1
740580,2022-11-21,25977F1E02F92D2B5AB3605B196C7ACB,37458AD74B59A3DE08EAE571C63DD3CF,7D1B71037D27D5597BEA1D5077663541,Возврат,1
740581,2022-11-21,126F1B83C0B42598DEE7E815A6ACAC71,DB81FBEFD3EAA20E18DCCCD397717499,126F1B83C0B42598DEE7E815A6ACAC71,Возврат,2
740582,2022-11-21,0E8EEC5192C68CF67F0E02AE30AFB56A,0504A3B05E1BD8A63D3A83F13CF49658,88747153B9CE8C25EA0347DC712C1368,Возврат,1


In [106]:
px.line(enter.groupby('dt').sum())
# px.line(enter.groupby('dt').sum().cumsum())

C:\Users\simple\AppData\Local\Temp\ipykernel_23780\1319583328.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [8]:
enter.groupby('operation_type').sum()

In [9]:
products = pd.read_csv('../data/marking/products.csv')

In [10]:
products

,gtin,inn,product_name,product_short_name,tnved,tnved10,brand,country,volume
0,8C414C7757D587886C10DB0CCEA641A2,D0B1FE981FCC19F934C3FFD91690430F,68F60FA530914522B26E25F262EBC6D6,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,58A4B52373651DA4292AD5725D388F8A,D2D1641B196DA5477D40C0907FD5F1DA,NaN,10
1,3D8C701836C4C0A5B33D57522A7EDBAB,9B0AFDA27FCCB188768FA9B611183387,670017E2D871B8962E5456CA71F6A1D1,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,58A4B52373651DA4292AD5725D388F8A,670017E2D871B8962E5456CA71F6A1D1,NaN,НЕ КЛАССИФИЦИРОВАНО
2,B1308C75C38993B0EEA22C8FE2899C2A,86CAE1EAD8AF200ED46453FD18FA1A04,97181A85E3400C38AB223F17A5DCCFCC,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,ABB51261177B26E23AC2DE0F0508D6AB,NaN,100
3,1D71E57CA201A8ACEADBAA8D6E7039DE,86CAE1EAD8AF200ED46453FD18FA1A04,EA5CEDF8FA0F5BEBD2F047151B2DBE86,B8AF2335E04E7BAD5168D9B39FABAC21,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,EDFFD44E0EC18764685E952553560EEF,NaN,100
4,5DEF4AA62907D322F85FA88839F00E8F,86CAE1EAD8AF200ED46453FD18FA1A04,75AF532015CFA92F71FF54ADEDB9BEA3,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,75AF532015CFA92F71FF54ADEDB9BEA3,NaN,50
...,...,...,...,...,...,...,...,...,...
393462,9205239C92C9E581328496DB794F60EC,8F599665A672F2CB6D29C23D4DAC7B9D,F8EE293A2AC51173AF97BE12876D6500,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,2B4A0053B03542072C3C13C9A3EC33E1,NaN,100
393463,AD775CC9CB0F39F9A5541C2CCE12E4E6,3B5BEA39AF78664AEA44752371845804,EE747123845C7B4F4EA96B3626538577,B8AF2335E04E7BAD5168D9B39FABAC21,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,A9EF1048419D8E2D4848D9389275CC0E,СОЕДИНЕННОЕ КОРОЛЕВСТВО,100
393464,5EEBF058F0AADCB4095A39E8EED63CF3,3B5BEA39AF78664AEA44752371845804,1056EF3A7F6831AFF867A3C072328983,B8AF2335E04E7BAD5168D9B39FABAC21,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,A9EF1048419D8E2D4848D9389275CC0E,СОЕДИНЕННОЕ КОРОЛЕВСТВО,100
393465,B54380CC5CCC0F3DDD2D448177ECB4BC,3B5BEA39AF78664AEA44752371845804,B84184EE78EB828BAE9C6C0324C0CBA6,B8AF2335E04E7BAD5168D9B39FABAC21,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,35DE9DA6A73429012DCB8B879A24E731,ФРАНЦИЯ,75


In [11]:
retailers = pd.read_csv('../data/marking/retailers.csv')

In [12]:
retailers

,id_sp_,inn,region_code,city_with_type,city_fias_id,postal_code
0,C72AC534DBF5E6369840FFE9DA9FF403,0056F9FCFA009E493BEFEAE4FAA71D82,51,NaN,NaN,184421.0
1,591942644AE25AE4DD26CF943BB97414,81AAC456284F455FC2B141E6E8F1A76C,66,г Екатеринбург,2763c110-cb8b-416a-9dac-ad28a55b4402,620027.0
2,A6E2261F16F84FF09E2B615EDB008943,87C2FDB73C26C96266A0947C2BB714BF,62,г Рязань,86e5bae4-ef58-4031-b34f-5e9ff914cd55,390026.0
3,DC3B647A089FA064DE61E0C778F5EE45,87C2FDB73C26C96266A0947C2BB714BF,62,г Рязань,86e5bae4-ef58-4031-b34f-5e9ff914cd55,390017.0
4,D2F125BAA947B6293CD055DA1DEEA87D,87C2FDB73C26C96266A0947C2BB714BF,62,г Рыбное,91d71d8c-2f1a-43c8-bc84-d78dc4245a1b,391112.0
...,...,...,...,...,...,...
58396,7230FEFB3B05186DD7C8DB90032BA110,AA7DCE37EB49B4A0CCF74C9FE1A1563B,42,г Прокопьевск,2d8b9d01-1c4f-4987-8a0b-1eb5080c1f38,653039.0
58397,43441D66745A34C6A358084B67EA9180,380E33F3782067D65B76C4B36EDC95E1,18,NaN,NaN,427088.0
58398,F296B3072969E65CB3F836A9DAE41671,7AA85B38D45C99C4143D6D30FC69A804,42,г Прокопьевск,2d8b9d01-1c4f-4987-8a0b-1eb5080c1f38,653010.0
58399,EC2ADD23B19CB66BE2B138F175BAAA25,E139E6A1A0F51E02261249F03E0A1402,25,г Владивосток,7b6de6a5-86d0-4735-b11a-499081111af8,690090.0


In [13]:
agents = pd.read_csv('../data/marking/agents.csv')

In [14]:
agents

,inn,region_code
0,60DFB6F87ABF6232A5D97A1A827F8D88,1
1,2D13C279C7CEC509D21695F0268ECCEA,1
2,1705C73792D7A1A5ABDF1883FE851004,1
3,B4CC35BD7D8F69E665160F548EBE125D,23
4,2DB318BEA4A44280FA488A0B2AF36254,1
...,...,...
23467,FD378AE7B1D03A5B949A96BF2715B941,77
23468,CB8661DA918DFA68498D7F929C426E1E,99
23469,92ECDE80D71E459551E809AF332785AA,77
23470,86B3B09285C227598C66DE143C5AD336,78


In [15]:
movement = pd.read_csv('../data/marking/movement.csv')

In [16]:
movement

,dt,gtin,prid,sender_inn,receiver_inn,cnt_moved
0,2021-11-22,867FDFE9E03925792F329DEDBAA3955F,9246A2BE043136FC4B4A166BB0E25D3E,2E55013259E009F4FC1E5EFCF170EA8B,8D5F8E7D20AD9A8D0E8EE5769321D8C3,1
1,2021-11-22,4FB53B945150E362C307D60297066562,9246A2BE043136FC4B4A166BB0E25D3E,32B53198A919C559CB84DC0487947E80,EDE6F6D21300D96A9E207EA4F59321EF,1
2,2021-11-22,38BA9B93C5C987770AD698646D2FAB66,2DADC5DF60A9FC194A3388C1612CC967,3A87113AC25EFD88BD3407ACECFA3869,D02A2294DDF3BCF7218D6FEFDB35A906,1
3,2021-11-22,FA77213DA058666C6781164754CBC0EB,FFC432B34974A68B2845FBED9A7DEB29,C5CDC4F467B7CDA0F3E082BFFEDA9938,051C7D77FE590C5428DB57FD48C5DB9C,3
4,2021-11-22,E5635C538FE51D88504CE5CFF8BC1D2C,6E536425E534ABA7AB9FEA2ACA91A775,AE7F5C67298A67CACD2A193C9405524F,0D28C0E0C8ADAA3D9B55B557C71D8A8E,2
...,...,...,...,...,...,...
8859253,2022-11-21,F3DE4F005EF65C8E66D7AB0683B54F54,7D1B71037D27D5597BEA1D5077663541,D18B1B470E57871FAB5B4E949E256BB4,08AF18664D85C4D2D22E042F7DD3266A,3
8859254,2022-11-21,C21ED5B431F287F33AD42C71C981099E,9246A2BE043136FC4B4A166BB0E25D3E,4671D4CBD5C6CC6315746B067D64C768,D531EF78F27C40B11E0266992ADFA856,1
8859255,2022-11-21,568E5BE2AFF72BC2F47BF0C5162A52DD,3F9C20F2F0335180915DD91E8F974882,40B7AB50AEFCAF8F2B5D47D7D57185BC,E34345C4111AAC1E88642C7A25543085,1
8859256,2022-11-21,5E0E70F49C286C2E181E17E5C00C9B07,F8E0B42035183C08A9821B76843B4D6A,050C148EBA7FD16960415F2084D22EA8,9AA1C8612DC529D5DC2844F9E0F684DC,1


In [17]:
px.line(movement.groupby('dt').sum())

C:\Users\simple\AppData\Local\Temp\ipykernel_23780\3025518870.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [18]:
leave = pd.read_csv('../data/marking/leave.csv')

In [19]:
leave

,dt,gtin,prid,inn,id_sp_,type_operation,price,cnt
0,2021-11-22,5A3E5F7B2D093D1D6CB3CF93BA9AC8A6,DC7432556B58D454B4E37F0F0D1E663D,B035ED5D308BC70E5EA9342B6DBF5810,19485F700068E6A9A8593AFDE9803638,Продажа конечному потребителю в точке продаж,3500,1
1,2021-11-22,9A2202A48FC5E462C44B78B0C79E1DBD,D18B1B470E57871FAB5B4E949E256BB4,DD486FED86564603D2CD7CF9EED2C458,C2969A4C8A40C12902EAD4936E303045,Продажа конечному потребителю в точке продаж,56714,1
2,2021-11-22,877ABE9DE14E0B510DFC8DE63EEFA817,62B5EAEF59E6693F44B1074C1140BF45,62B5EAEF59E6693F44B1074C1140BF45,5637334F301773E28AE8730713E6D5F7,Продажа конечному потребителю в точке продаж,74314,1
3,2021-11-22,B004F889F17F8AB365C494B3D0D06E1A,D7326D87A37D73AF19AF53668A7116D6,D7326D87A37D73AF19AF53668A7116D6,0FF56DEC16050D3B6372F2CB1BD6DFCB,Продажа конечному потребителю в точке продаж,4999,1
4,2021-11-22,E1C0503EA9201D56D7173CB4CA4A6AFA,50955BB0B97298EEAEB1F6CF9712F37A,39FFA592BE77CA7DE37C65C1B8423E8C,9E660AA1DE428BA88D1942DB2824BDAB,Продажа конечному потребителю в точке продаж,29857,1
...,...,...,...,...,...,...,...,...
52095176,2022-04-07,5D74B67E1DAC42C7526F27A6BDA73FF6,DA62EC79660CF21AC37A260DA6F642C4,E147E089C99F3A5EFEEF57739F60EFD1,09C6904B8ACA75E3380000EB4AC0A616,Продажа конечному потребителю в точке продаж,1128,2
52095177,2022-04-07,B1C38C41D23BA4AC631FCF073744A362,D80CBF60BE36ACE344F8350B648C688D,E147E089C99F3A5EFEEF57739F60EFD1,NaN,Продажа конечному потребителю в точке продаж,3571,1
52095178,2022-04-07,2C654E2727AACB613F30DB2FF30844A1,6BC47888CB4E777EE8DF0B940FFA18C7,E147E089C99F3A5EFEEF57739F60EFD1,2FF1ACF094C477DF28C6059CA11424BF,Продажа конечному потребителю в точке продаж,4196,1
52095179,2022-04-07,859C541431985132C29F457945761242,DA62EC79660CF21AC37A260DA6F642C4,E147E089C99F3A5EFEEF57739F60EFD1,07905E35AB800714D847D6DEBAE46F47,Продажа конечному потребителю в точке продаж,985,3


In [20]:
# leave.groupby('type_operation').sum()

In [21]:
px.line(leave.groupby('dt').sum())

C:\Users\simple\AppData\Local\Temp\ipykernel_23780\3373305044.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [22]:
# leave.groupby('gtin').sum()

In [23]:
# leave[leave['type_operation'] == 'Конечная продажа организации']['price'].mean()
# # Продажа конечному потребителю в точке продаж // mean = 29494
# # Дистанционная продажа конечному потребителю // mean = 406
# # Конечная продажа организации // 450


In [24]:
# leave[leave['price'] == 0]['cnt'].sum()

In [25]:
# leave[leave['price'] != 0]['cnt'].sum()

In [26]:
# anomalies = leave[leave['price'] != 0].groupby(by='gtin').std(numeric_only=True)
# anomalies

In [27]:
# enter1 = enter
# # enter1 = enter[enter['operation_type'] == 'РФ']
# enter1.groupby('prid').count()

# dft1 = enter1[enter1['dt'] < '2021-12-01'].groupby('prid').sum()
# dft2 = enter1[enter1['dt'] >= '2021-12-01'][enter1['dt'] < '2022-01-01'].groupby('prid').sum()
# dft3 = enter1[enter1['dt'] >= '2022-01-01'][enter1['dt'] < '2022-02-01'].groupby('prid').sum()
# dft4 = enter1[enter1['dt'] >= '2022-02-01'][enter1['dt'] < '2022-03-01'].groupby('prid').sum()
# dft5 = enter1[enter1['dt'] >= '2022-03-01'][enter1['dt'] < '2022-04-01'].groupby('prid').sum()
# dft6 = enter1[enter1['dt'] >= '2022-04-01'][enter1['dt'] < '2022-05-01'].groupby('prid').sum()
# dft7 = enter1[enter1['dt'] >= '2022-05-01'][enter1['dt'] < '2022-06-01'].groupby('prid').sum()
# dft8 = enter1[enter1['dt'] >= '2022-06-01'][enter1['dt'] < '2022-07-01'].groupby('prid').sum()
# dft9 = enter1[enter1['dt'] >= '2022-07-01'][enter1['dt'] < '2022-08-01'].groupby('prid').sum()
# dft10 = enter1[enter1['dt'] >= '2022-08-01'][enter1['dt'] < '2022-09-01'].groupby('prid').sum()
# dft11 = enter1[enter1['dt'] >= '2022-09-01'][enter1['dt'] < '2022-10-01'].groupby('prid').sum()
# dft12 = enter1[enter1['dt'] >= '2022-10-01'].groupby('prid').sum()

In [28]:
# dfq = dft1
# for dft in [dft2,dft3,dft4,dft5,dft6,dft7,dft8,dft9,dft10,dft11,dft12,enter1.groupby('prid').sum()]:
#     dfq = dfq.merge(dft, left_on='prid', right_on='prid', how='outer')
# dfq.fillna(0)

In [29]:
# leave1 = leave.drop(['inn', 'prid', 'type_operation', 'gtin', 'price'], axis=1)
# retailers1 = retailers.drop(['postal_code'], axis=1)
#
# dft1 = leave1[leave1['dt'] < '2021-12-01'].merge(retailers1).groupby('region_code').sum()
# dft2 = leave1[leave1['dt'] >= '2021-12-01'][leave1['dt'] < '2022-01-01'].merge(retailers1).groupby('region_code').sum()
# dft3 = leave1[leave1['dt'] >= '2022-01-01'][leave1['dt'] < '2022-02-01'].merge(retailers1).groupby('region_code').sum()
# dft4 = leave1[leave1['dt'] >= '2022-02-01'][leave1['dt'] < '2022-03-01'].merge(retailers1).groupby('region_code').sum()
# dft5 = leave1[leave1['dt'] >= '2022-03-01'][leave1['dt'] < '2022-04-01'].merge(retailers1).groupby('region_code').sum()
# dft6 = leave1[leave1['dt'] >= '2022-04-01'][leave1['dt'] < '2022-05-01'].merge(retailers1).groupby('region_code').sum()
# dft7 = leave1[leave1['dt'] >= '2022-05-01'][leave1['dt'] < '2022-06-01'].merge(retailers1).groupby('region_code').sum()
# dft8 = leave1[leave1['dt'] >= '2022-06-01'][leave1['dt'] < '2022-07-01'].merge(retailers1).groupby('region_code').sum()
# dft9 = leave1[leave1['dt'] >= '2022-07-01'][leave1['dt'] < '2022-08-01'].merge(retailers1).groupby('region_code').sum()
# dft10 = leave1[leave1['dt'] >= '2022-08-01'][leave1['dt'] < '2022-09-01'].merge(retailers1).groupby('region_code').sum()
# dft11 = leave1[leave1['dt'] >= '2022-09-01'][leave1['dt'] < '2022-10-01'].merge(retailers1).groupby('region_code').sum()
# dft12 = leave1[leave1['dt'] >= '2022-10-01'].merge(retailers1).groupby('region_code').sum()
# dft_all = leave1.merge(retailers1).groupby('region_code').sum()

In [30]:
# dfq = dft1
# for dft in [dft2, dft3, dft4, dft5, dft6, dft7, dft8, dft9, dft10, dft11, dft12, dft_all]:
#     dfq = dfq.merge(dft, left_on='region_code', right_on='region_code', how='outer')
# dfq.fillna(0)

In [31]:
# enter3 = enter.drop([], axis=1)
# agents1 = agents.drop([], axis=1)
#
# dft1 = enter3[enter3['dt'] < '2021-12-01'].merge(agents1).groupby('region_code').sum()
# dft2 = enter3[enter3['dt'] >= '2021-12-01'][enter3['dt'] < '2022-01-01'].merge(agents1).groupby('region_code').sum()
# dft3 = enter3[enter3['dt'] >= '2022-01-01'][enter3['dt'] < '2022-02-01'].merge(agents1).groupby('region_code').sum()
# dft4 = enter3[enter3['dt'] >= '2022-02-01'][enter3['dt'] < '2022-03-01'].merge(agents1).groupby('region_code').sum()
# dft5 = enter3[enter3['dt'] >= '2022-03-01'][enter3['dt'] < '2022-04-01'].merge(agents1).groupby('region_code').sum()
# dft6 = enter3[enter3['dt'] >= '2022-04-01'][enter3['dt'] < '2022-05-01'].merge(agents1).groupby('region_code').sum()
# dft7 = enter3[enter3['dt'] >= '2022-05-01'][enter3['dt'] < '2022-06-01'].merge(agents1).groupby('region_code').sum()
# dft8 = enter3[enter3['dt'] >= '2022-06-01'][enter3['dt'] < '2022-07-01'].merge(agents1).groupby('region_code').sum()
# dft9 = enter3[enter3['dt'] >= '2022-07-01'][enter3['dt'] < '2022-08-01'].merge(agents1).groupby('region_code').sum()
# dft10 = enter3[enter3['dt'] >= '2022-08-01'][enter3['dt'] < '2022-09-01'].merge(agents1).groupby('region_code').sum()
# dft11 = enter3[enter3['dt'] >= '2022-09-01'][enter3['dt'] < '2022-10-01'].merge(agents1).groupby('region_code').sum()
# dft12 = enter3[enter3['dt'] >= '2022-10-01'].merge(agents1).groupby('region_code').sum()
# dft_all = enter3.merge(agents1).groupby('region_code').sum()

In [32]:
# dfq = dft1
# for dft in [dft2, dft3, dft4, dft5, dft6, dft7, dft8, dft9, dft10, dft11, dft12, dft_all]:
#     dfq = dfq.merge(dft, left_on='region_code', right_on='region_code', how='outer')
# dfq.fillna(0)

In [33]:
# enter2 = enter
# # enter2 = enter[enter['operation_type'] == 'РФ']
# enter2.groupby('prid').count()
#
# dft1 = enter2[enter2['dt'] < '2021-12-01'].sum()
# dft2 = enter2[enter2['dt'] >= '2021-12-01'][enter2['dt'] < '2022-01-01'].sum()
# dft3 = enter2[enter2['dt'] >= '2022-01-01'][enter2['dt'] < '2022-02-01'].sum()
# dft4 = enter2[enter2['dt'] >= '2022-02-01'][enter2['dt'] < '2022-03-01'].sum()
# dft5 = enter2[enter2['dt'] >= '2022-03-01'][enter2['dt'] < '2022-04-01'].sum()
# dft6 = enter2[enter2['dt'] >= '2022-04-01'][enter2['dt'] < '2022-05-01'].sum()
# dft7 = enter2[enter2['dt'] >= '2022-05-01'][enter2['dt'] < '2022-06-01'].sum()
# dft8 = enter2[enter2['dt'] >= '2022-06-01'][enter2['dt'] < '2022-07-01'].sum()
# dft9 = enter2[enter2['dt'] >= '2022-07-01'][enter2['dt'] < '2022-08-01'].sum()
# dft10 = enter2[enter2['dt'] >= '2022-08-01'][enter2['dt'] < '2022-09-01'].sum()
# dft11 = enter2[enter2['dt'] >= '2022-09-01'][enter2['dt'] < '2022-10-01'].sum()
# dft12 = enter2[enter2['dt'] >= '2022-10-01'].sum()
# dft_all = enter2.sum()

In [34]:
# dfq = dft1
# for dft in [dft2,dft3,dft4,dft5,dft6,dft7,dft8,dft9,dft10,dft11,dft12,dft_all]:
#     dfq = dfq.merge(dft, left_on='prid', right_on='prid', how='outer')
# dfq.fillna(0)

In [35]:
# leave4 = leave[leave['type_operation'] == 'Списание / Вывод из оборота без получателя']
#
# dft1 = leave4[leave4['dt'] < '2021-12-01'].groupby(by=lambda x: 1).sum()
# dft2 = leave4[leave4['dt'] >= '2021-12-01'][leave4['dt'] < '2022-01-01'].groupby(by=lambda x: 1).sum()
# dft3 = leave4[leave4['dt'] >= '2022-01-01'][leave4['dt'] < '2022-02-01'].groupby(by=lambda x: 1).sum()
# dft4 = leave4[leave4['dt'] >= '2022-02-01'][leave4['dt'] < '2022-03-01'].groupby(by=lambda x: 1).sum()
# dft5 = leave4[leave4['dt'] >= '2022-03-01'][leave4['dt'] < '2022-04-01'].groupby(by=lambda x: 1).sum()
# dft6 = leave4[leave4['dt'] >= '2022-04-01'][leave4['dt'] < '2022-05-01'].groupby(by=lambda x: 1).sum()
# dft7 = leave4[leave4['dt'] >= '2022-05-01'][leave4['dt'] < '2022-06-01'].groupby(by=lambda x: 1).sum()
# dft8 = leave4[leave4['dt'] >= '2022-06-01'][leave4['dt'] < '2022-07-01'].groupby(by=lambda x: 1).sum()
# dft9 = leave4[leave4['dt'] >= '2022-07-01'][leave4['dt'] < '2022-08-01'].groupby(by=lambda x: 1).sum()
# dft10 = leave4[leave4['dt'] >= '2022-08-01'][leave4['dt'] < '2022-09-01'].groupby(by=lambda x: 1).sum()
# dft11 = leave4[leave4['dt'] >= '2022-09-01'][leave4['dt'] < '2022-10-01'].groupby(by=lambda x: 1).sum()
# dft12 = leave4[leave4['dt'] >= '2022-10-01'].groupby(by=lambda x: 1).sum()
# dft_all = leave4.groupby(by=lambda x: 1).sum()

In [36]:
# dfq = dft1
# for dft in [dft2,dft3,dft4,dft5,dft6,dft7,dft8,dft9,dft10,dft11,dft12,dft_all]:
#     dfq = dfq.merge(dft, how='outer')
# dfq.fillna(0)

In [102]:
timestamps = [str(a) for a in pd.date_range('2021-11-21', '2022-11-21', freq='M')]
# timestamps = [str(a) for a in pd.date_range('2021-11-21', '2022-11-21', freq='W')]
ranges = list(zip(timestamps, timestamps[1:]))

In [56]:
def drawIncoming():
    px.line(enter.groupby('dt').sum()).show()
    # px.line(enter[enter['operation_type'] == 'РФ'].groupby('dt').sum().cumsum()).show()
drawIncoming()

C:\Users\simple\AppData\Local\Temp\ipykernel_23780\2006279051.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [ ]:
def drawOutgoing():
    px.line(leave.groupby('dt').sum()).show()
    # px.line(leave[leave['type_operation'] == 'Продажа конечному потребителю в точке продаж'].groupby('dt').sum().cumsum()).show()
drawOutgoing()

In [103]:
# @DOC: Ввод по типам операций
def calcProduction():
    base = enter.drop(['inn', 'gtin', 'prid'], axis=1)
    accum = base.iloc[0:0].drop(['cnt'], axis=1)
    for [a, b] in ranges:
        step = base[base['dt'] >= a][base['dt'] < b].groupby('operation_type').sum(numeric_only=True)
        accum = accum.merge(step, left_on='operation_type', right_on='operation_type', how='outer')
    accum = accum.fillna(0)
    accum = accum.drop(['dt'], axis=1)
    return accum
production = calcProduction()
production

C:\Users\simple\AppData\Local\Temp\ipykernel_23780\3270544586.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\simple\AppData\Local\Temp\ipykernel_23780\3270544586.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\simple\AppData\Local\Temp\ipykernel_23780\3270544586.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\simple\AppData\Local\Temp\ipykernel_23780\3270544586.py:7: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.

C:\Users\simple\AppData\Local\Temp\ipykernel_23780\3270544586.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\simple\AppData\Local\Temp\ipykernel_23780\3270544586.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\simple\AppData\Local\Temp\ipykernel_2378

,operation_type,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,...,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y
0,Возврат,10930.0,7081,10333.0,14152.0,15963,11135,3986.0,18035.0,13164,...,12034,10457,7680,9359,133332,25780,255696,162839,9698,154029
1,Импорт. ЕАЭС,87739.0,41118,89261.0,54456.0,90735,88164,8650.0,30407.0,79935,...,68378,187087,271611,41724,114265,288067,99259,91316,84932,191927
2,Импорт. Не ЕАЭС,1619960.0,2251191,1352534.0,1815351.0,1988177,912459,460351.0,1761455.0,886041,...,659143,666836,730983,727452,1068621,1382015,537727,1180137,943735,1112623
3,Маркировка остатков,507248.0,203186,7999.0,15079.0,10827,12765,4091.0,15977.0,10699,...,2184,26328,209061,221,251,202,194,176,203,225
4,Перемаркировка,24183.0,38828,20457.0,21611.0,24346,21152,3547.0,14726.0,14021,...,16251,11915,10212,27382,14717,24081,28854,13903,21943,23560
5,РФ,1759941.0,1556184,1695195.0,1746945.0,1607046,1004312,425167.0,1771935.0,2050052,...,1084620,1313014,1533878,1374729,1395870,1160408,1418544,1280786,1595395,1492995
6,Принято от физического лица,0.0,1,0.0,0.0,23,48,0.0,0.0,1,...,2200,1294,229,558,866,428,760,681,429,776


In [104]:
# @DOC: Вывод по типам операций
def calcConsumption():
    base = leave.drop(['inn', 'gtin', 'prid', 'id_sp_', 'price'], axis=1)
    accum = base.iloc[0:0].drop(['cnt'], axis=1)
    for [a, b] in ranges:
        step = base[base['dt'] >= a][base['dt'] < b].groupby('type_operation').sum(numeric_only=True)
        accum = accum.merge(step, left_on='type_operation', right_on='type_operation', how='outer')
    accum = accum.fillna(0)
    accum = accum.drop(['dt'], axis=1)
    return accum
consumption = calcConsumption()
consumption

C:\Users\simple\AppData\Local\Temp\ipykernel_23780\1054091358.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\simple\AppData\Local\Temp\ipykernel_23780\1054091358.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\simple\AppData\Local\Temp\ipykernel_23780\1054091358.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\simple\AppData\Local\Temp\ipykernel_23780\1054091358.py:7: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.

C:\Users\simple\AppData\Local\Temp\ipykernel_23780\1054091358.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\simple\AppData\Local\Temp\ipykernel_23780\1054091358.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\simple\AppData\Local\Temp\ipykernel_2378

,type_operation,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,...,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y
0,Дистанционная продажа конечному потребителю,223540.0,268894.0,258892.0,232204.0,238682.0,205159.0,105851.0,145096.0,146183.0,...,210866.0,235936.0,158895.0,175776.0,319314.0,467978.0,191176.0,250737.0,231018.0,234854.0
1,Конечная продажа организации,60113.0,39272.0,38618.0,46134.0,66076.0,22201.0,3515.0,16466.0,45668.0,...,5470.0,6891.0,10691.0,10951.0,5522.0,6208.0,26680.0,19711.0,9752.0,12194.0
2,Продажа конечному потребителю в точке продаж,1979952.0,1991730.0,2189524.0,2630298.0,3117279.0,3784528.0,1238671.0,1361705.0,1360691.0,...,1419651.0,1262211.0,1156850.0,1376748.0,1608894.0,1336852.0,1353908.0,1536954.0,1483562.0,1710346.0
3,Продажи за пределы РФ,621821.0,314315.0,599752.0,553334.0,198589.0,262726.0,87269.0,381701.0,470083.0,...,316069.0,191628.0,214539.0,266419.0,259559.0,199426.0,226379.0,121629.0,226717.0,205555.0
4,Прочий тип вывода из оборота,25727.0,23805.0,26446.0,34280.0,46261.0,57008.0,19053.0,18432.0,18248.0,...,41718.0,38913.0,35775.0,40369.0,44861.0,43081.0,43406.0,44935.0,32974.0,37751.0
5,Списание / Вывод из оборота без получателя,7263.0,15662.0,9755.0,31523.0,15673.0,8186.0,1056.0,13497.0,15522.0,...,13034.0,16320.0,24914.0,27573.0,76208.0,61659.0,27381.0,24272.0,24656.0,21021.0
6,Продажа по государственному контракту,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
